In [1]:
from dataAnalysis import *
from myKriging import *
from draw import *
np.set_printoptions(threshold=100)


In [6]:
# data = get_data("diercimonibeifen_31.xlsx", sheet_name_template="idx=z", usecols=("B:CW"), z_size=100, y_size=100)
# data = get_data("basicflairbeifen_21.xlsx", sheet_name_template="idx=z", usecols=("B:CW"), z_size=100, y_size=100)
data = get_data("../PINN/DATA.xlsx")

# 分析data数据的统计信息
all_targets = [data[i][j][k] for i in range(len(data.keys())) for j in range(len(data[0][1])) for k in range(len(data[0][0]))]
variance = np.var(all_targets)
minValue = np.min([i for i in all_targets if i!=0])
maxValue = np.max(all_targets)

print("data方差为:",variance,"\ndata最大值为:",maxValue,"\ndata最小值为:",minValue)

data方差为: 4725203.880520142 
data最大值为: 1574264.0 
data最小值为: 34.8256


In [7]:
# # data[z][x][y]
# data[5][48][45] #桶源

# data[54][97][90] #墙壁源

In [11]:
# z,y,x
df_training = training_sampling(data, 48, 45, 5, 30, #在box模式下没用，但是要加
                                step_sizes=[5], 
                                x_y_reverse = True,
                                use_box_area = True, #是否使用固定空间来进行取样, 设置了这个后上面的(48, 45, 5, 30就无效了)
                                use_box = [[5,5,5],90,90,60], #左前下点，x方向延申长度，y方向延申长度，z方向延申长度 
                                direction = "3vector")
# df_training = training_sampling(data, 63, 58, 29, 28, step_sizes=[5], x_y_reverse = True, direction = "3vector")
# df_training = training_sampling(data, 0, 0, 0 , 99 ,step_sizes=[15], x_y_reverse = False, direction = "3vector")
df_training.drop_duplicates(inplace=True)  # 删除重复行
df_training.reset_index(drop=True, inplace=True)  # 重新设置索引
df_training

,x,y,z,target
0,5,5,5,45.56526
1,5,5,10,46.43686
2,5,5,15,48.00318
3,5,5,20,48.10044
4,5,5,25,48.74888
...,...,...,...,...
3933,95,90,20,131.98590
3934,95,95,5,97.96942
3935,95,95,10,104.03710
3936,95,95,15,110.38430


In [12]:
df_testing = testing_sampling(data, 48, 45, 5, 30, 
                              step_sizes=[3], 
                              direction = "3vector", 
                              use_box_area = True, #是否使用固定空间来进行取样
                              use_box = [[5,5,5],90,90,60], #左前下点，x方向延申长度，y方向延申长度，z方向延申长度 
                              sample_like_training = True)
# df_testing = testing_sampling(data, 0, 0, 0, 99, step_sizes=[3], direction = "3vector", sample_like_training = True)
# df_testing = testing_sampling(data, 63, 58, 29, 28, step_sizes=[1], direction = "3vector")
df_testing.drop_duplicates(inplace=True)  # 删除重复行
df_testing.reset_index(drop=True, inplace=True)  # 重新设置索引
df_testing

,x,y,z,target
0,5,5,5,45.56526
1,5,5,8,46.35245
2,5,5,11,46.56773
3,5,5,14,47.06169
4,5,5,17,48.19140
...,...,...,...,...
16860,95,95,11,105.04230
16861,95,95,14,108.29130
16862,95,95,17,111.22720
16863,95,95,20,114.95710


In [13]:
# variogram_model指定方差模型(之后可以比较使用不同种类GPU加速的速度差异(例如cupy,pycuda重写和其他)，以及不同部分的时间消耗对比)
model = training(df_training, 
                 variogram_model="exponential", 
                 nlags = 20, 
                 enable_plotting=False, 
                 weight=True, 
                 uk=False)

# 以下时间总和对不上是因为打印耗时，到后面部分就对上了，因为打印耗时占比很小。
# 计算平方距离矩阵的时间为 3.218650817871094e-05
# 设置krige矩阵和RHS（右端向量）的时间为 3.933906555175781e-05
# 求解的时间为 7.05718994140625e-05
# 第三-1-1部分计时为 0.00016808509826660156


init消耗时间为1.02



In [ ]:

model_cpu = training(df_training, 
                 variogram_model="exponential", 
                 nlags = 20, 
                 enable_plotting=False, 
                 weight=True, 
                 uk=False,
                 cpu_on = True)
# cpu不分块不多线程
testing_value_cpu = testing(df_testing, 
                            model_cpu, 
                            style = "cpu_nb")

In [ ]:
# cpu分块不多线程
testing_value_cpu_fenkuai = testing(df_testing, 
                                    model, 
                                    block_size = 20000, 
                                    cpu_on = True, 
                                    multi_process=False)

In [ ]:
# cpu分块多线程
testing_value_cpu_fenkuai = testing(df_testing, 
                                    model, 
                                    block_size = 30000, 
                                    cpu_on = True, 
                                    multi_process=True)

In [14]:
# gpu分块不多进程
testing_value_gpu_fenkuai = testing(df_testing, 
                                    model, 
                                    block_size = 50000, 
                                    cpu_on = False, 
                                    multi_process=False,
                                    print_time = True)

输入的检验矩阵中有 0 个0（gpukriging自身检验，外部程序不作考虑）
_get_kriging_matrix消耗时间为0.29

Block 0 processing time: 0.51 seconds
execute消耗时间为0.81

模型的平均相对误差2为 0.007588976792836991


In [ ]:
# 生成GPU vs CPU精度对比图
try:
    # 确保有CPU模型（如果之前没有训练过）
    if 'model_cpu' not in locals():
        print("训练CPU模型...")
        model_cpu = training(df_training, 
                            variogram_model="exponential", 
                            nlags = 20, 
                            enable_plotting=False, 
                            weight=True, 
                            uk=False,
                            cpu_on = True)
    
    # 为了减少绘图数据量，随机采样部分测试点（可选）
    sample_size = min(5000, len(df_testing))  # 最多5000个点用于绘图
    if len(df_testing) > sample_size:
        df_testing_plot = df_testing.sample(n=sample_size, random_state=42).reset_index(drop=True)
        print(f"为加速绘图，随机采样了{sample_size}个测试点")
    else:
        df_testing_plot = df_testing
        print(f"使用全部{len(df_testing)}个测试点绘图")
    
    # 调用绘图函数
    print("开始生成精度对比图...")
    draw_result3D(data=df_testing_plot, 
                  model=model, 
                  model_cpu=model_cpu,
                  draw_source=False,  # 是否绘制源点
                  vol_size=1,
                  source={"x": 48, "y": 45, "z": 5})  # 源点坐标
    
    print("精度对比图生成完成！图片已保存为 precision.pdf")
    
except Exception as e:
    print(f"绘图过程中出现错误: {e}")
    print("可能的解决方案：")
    print("1. 检查model和model_cpu是否都已训练完成")
    print("2. 检查df_testing数据格式是否正确")
    print("3. 确保所有依赖库都已正确导入")
